<a href="https://colab.research.google.com/github/gyulgyulyan/MyScripts/blob/main/ROAS150_30d_em_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from scipy.misc import derivative
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import re
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier, plot_metric
import lightgbm as lgb
from scipy.misc import derivative
import numpy as np
from scipy import optimize
from scipy import special
import json
import gc
from pathlib import Path
from sklearn.metrics import accuracy_score
from statsmodels.regression.quantile_regression import QuantReg as qr
from scipy.optimize import leastsq

from sqlalchemy import create_engine # !pip install clickhouse-sqlalchemy

In [ ]:
df_merged=pd.read_csv('ml_em_jan_new.csv')

In [ ]:
quint=pd.read_csv('train_quantile.csv')
df_merged=pd.merge(df_merged, quint, on=["app_id", "country_installs"], how='left')

In [ ]:
df_merged['install_date_utc']=pd.to_datetime(df_merged['install_date_utc'])
df_merged=df_merged.loc[df_merged.device_id!='00000000-0000-0000-0000-000000000000']

#### PRE-FIT

In [ ]:
# фильтрация данных по максимальной дате входа и инапным платежам
devices = df_merged.loc[(df_merged['app_revenue_0_30'] == 0) & (df_merged['sub_price_usd'] > 0), 'device_id'].tolist()
df_merged.drop(df_merged[((df_merged['device_id'].isin(devices)) & (df_merged['app_revenue_0_30'] == 0)) | ((df_merged.max_das>30) & (df_merged.first_session<'2021-01-01'))].index, inplace = True)

In [ ]:
df_merged['last_session'] = pd.to_datetime(df_merged['last_session'], format='%Y-%m-%d')
df_merged['max_date'] = df_merged['install_date_utc'] + dt.timedelta(30) 
df_merged['days_from_last_session'] = df_merged['max_date']-df_merged['last_session']
df_merged['days_from_last_session'] = df_merged['days_from_last_session'].dt.days

df_merged['days_from_last_session']=np.where(df_merged.days_from_last_session<0, 0, df_merged.days_from_last_session)
df_merged['days_from_last_session']=np.where(df_merged.days_from_last_session>30, 30, df_merged.days_from_last_session)

df_merged['daylife'] = df_merged['last_session']-df_merged['install_date_utc']
df_merged['daylife'] = df_merged['daylife'].dt.days
df_merged['daylife']=np.where((df_merged.max_das<30) & ((df_merged.daylife<0) | (df_merged.daylife>30)),df_merged.max_das_0_30, 30-df_merged.days_from_last_session)

In [ ]:
df_merged['quan_20_num_rebill']=np.where(df_merged.quan_20_num_rebill>1,df_merged.quan_20_num_rebill,1)
df_merged['quan_20_num_dial']=np.where(df_merged.quan_20_num_dial>1,df_merged.quan_20_num_dial,1)

df_merged['r_score']=np.where(df_merged.days_from_last_session<=df_merged.quan_20_last_sess, 3,
                             np.where(df_merged.days_from_last_session<=df_merged.quan_80_last_sess, 2,1))
df_merged['f_score']=np.where(df_merged.num_sessions>df_merged.quan_80_sess, 3,
                             np.where(df_merged.num_sessions>1, 2,1))
df_merged['m_score']=np.where(df_merged.app_revenue_0_30>df_merged.quan_80_app_rev_0_30, 3,
                             np.where(df_merged.app_revenue_0_30>0, 2,1))
df_merged['das_score']=np.where(df_merged.num_rebills_0_30>=df_merged.quan_80_num_rebill, 3,
                             np.where(df_merged.num_rebills_0_30>=df_merged.quan_20_num_rebill, 2,1))
df_merged['dial_score']=np.where(df_merged.num_dialogs>=df_merged.quan_80_num_dial, 3,
                             np.where(df_merged.num_dialogs>=df_merged.quan_20_num_dial, 2,1))

In [ ]:
df_merged['rfm'] = df_merged['r_score'].map(str) + df_merged['f_score'].map(str) + df_merged['m_score'].map(str) + df_merged['das_score'].map(str) + df_merged['dial_score'].map(str)
df_merged['rfm'] = df_merged['rfm'].astype('category')

In [ ]:
vvp=pd.read_csv('vvp.csv', sep=";", low_memory=False)
df_merged=pd.merge(df_merged,vvp, how='left')
df_merged.loc[pd.isna(df_merged.vvp)==True,'vvp']=int(vvp.loc[vvp.country_profile=='unknown','vvp'])

In [ ]:
df_merged['diff_coun1']=np.where(df_merged['country_installs']==df_merged['country_signup'],1,0)
df_merged['diff_coun2']=np.where(df_merged['country_installs']==df_merged['country_profile'],1,0)
df_merged['diff_coun3']=np.where(df_merged['country_signup']==df_merged['country_profile'],1,0)

#### ДАТАСЕТЫ ДЛЯ КЛАССИФИКАТОРА

In [ ]:

cat_features = ['country_tier_cat', 'top_gender_cat',
       'campaign_type_cat', 'media_source_cat', 'country_installs_cat',
       'country_profile_cat','country_signup_cat', 'source_of_payment_cat', 'placement_id_cat',
        'sub_currency_cat',
       'purchase_currency_cat', 'trial_currency_cat',
       'reason_of_cancelling_cat','rfm','language_cat'
                ]

features = ['uid_count', 'country_count', 'city_count',
       'app_version_count', 'fs_visit_count', 'fs_visit_count_7_13',
       'fs_visit_count_10_13', 'fs_visit_count_13', 'fs_visit_count_12',
       'fs_visit_count_11', 'is_signup', 'is_start_trial', 'das_count_0',
       'das_count_1_3', 'das_count_4_7', 'das_count_7_13', 'das_count_0_13',
        
        'ad_revenue_0_13', 'ad_revenue_0_30',
       'top_age_group_', 'country_qol_idx', 'country_col_idx',
       'is_capital',
            
            'purchase_revenue_0', 'purchase_revenue_0_1','purchase_revenue_4', 'purchase_revenue_5', 'purchase_revenue_6',
       'purchase_revenue_7', 'purchase_revenue_8', 'purchase_revenue_9',
        'purchase_revenue_11', 'purchase_revenue_12',
       'purchase_revenue_13', 'purchase_revenue_15', 'purchase_revenue_20',
       'purchase_revenue_21', 'purchase_revenue_25', 'purchase_revenue_30',
       'purchase_revenue_0_13', 'purchase_revenue_0_30',
            
        'subscribe_revenue_0', 'subscribe_revenue_0_1', 'subscribe_revenue_1',
       'subscribe_revenue_2', 'subscribe_revenue_3', 'subscribe_revenue_4',
       'subscribe_revenue_5', 'subscribe_revenue_6', 'subscribe_revenue_7',
       'subscribe_revenue_8', 'subscribe_revenue_9','subscribe_revenue_10', 'subscribe_revenue_11', 'subscribe_revenue_12',
       'subscribe_revenue_13', 'subscribe_revenue_15', 'subscribe_revenue_20',
       'subscribe_revenue_21', 'subscribe_revenue_25', 'subscribe_revenue_30',
       'subscribe_revenue_0_30',
            
        'app_revenue_0', 'app_revenue_0_1',
       'app_revenue_1', 'app_revenue_2', 'app_revenue_3', 'app_revenue_4',
       'app_revenue_5', 'app_revenue_6', 'app_revenue_7', 'app_revenue_8',
       'app_revenue_9', 'app_revenue_10', 'app_revenue_11', 'app_revenue_12',
       'app_revenue_13', 'app_revenue_15', 'app_revenue_20', 'app_revenue_21',
       'app_revenue_25', 'app_revenue_30', 'app_revenue_0_30',  
            
        'ltv_0', 'ltv_1', 'ltv_2', 'ltv_3', 'ltv_4',
       'ltv_5', 'ltv_6', 'ltv_7', 'ltv_8', 'ltv_9', 'ltv_10', 'ltv_11',
        'ltv_12', 'ltv_13', 'ltv_15', 'ltv_20', 'ltv_21', 'ltv_25', 'ltv_30',
            
        'retention_0', 'retention_1', 'retention_2',
       'retention_3', 'retention_4', 'retention_5', 'retention_6',
       'retention_7', 'retention_8', 'retention_9', 'retention_10',
       'retention_11', 'retention_12', 'retention_13',
            
       'sub_price_usd', 'trial_price_usd', 'purchase_price_usd',
       'fs_sub_number', 'fs_sub_number_bool','entered_grace_period',
       'entered_grace_period_bool',  
            
        'has_cancel', 'cancel_after_trial',
       'sub_revenue', 'purchase_revenue', 'trial_revenue',
            'has_only_sub',
       'has_only_purchase', 'has_only_trial', 'has_purchases',
       'num_rebills_0_30',
            
        'num_sub_clicks', 'num_sub_watched', 'conversion_to_clicks', 
            
        'num_sessions_0',
       'num_sessions_1', 'num_sessions_2', 'num_sessions_3', 'num_sessions_4',
       'num_sessions_5', 'num_sessions_6', 'num_sessions_7', 'num_sessions_8',
        'num_sessions_9', 'num_sessions_10', 'num_sessions_11',
       'num_sessions_12', 'num_sessions_13', 'num_sessions_0_13',
       'num_sessions',
            
        'sum_session_duration', 'num_ads_0', 'num_ads_1', 'num_ads_2',
       'num_ads_3', 'num_ads_4', 'num_ads_5', 'num_ads_6', 'num_ads_7',
       'num_ads_8', 'num_ads_9', 'num_ads_10', 'num_ads_11', 'num_ads_12',
       'num_ads_13', 'num_ads', 'num_ads_0_13', 'num_dialogs',
       'num_dialogs_0_13', 'photo_views', 'photo_views_0_13',
       'ads_clicked_0_13', 'num_week', 'num_week_trial',  'vvp','max_das_0_30','days_from_last_session',
         'daylife' ,'diff_coun1','diff_coun2','diff_coun3',
            'num_input_mes','num_like','num_dislike','num_photo_upload','num_ban','num_send_mes',
            'r_score','f_score','m_score','das_score','dial_score'
           ]

In [ ]:
df_merged['has_visit_30_150']=np.where(df_merged.max_das>30,1,0)

#### ПРЕДИКТЫ КЛАССИФИКАТОРОВ

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def focal_loss_lgb(y_pred, dataset, alpha, gamma):
    y_true = dataset.get_label()
    a,g = alpha, gamma
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess

def focal_loss_lgb2(y_true,y_pred, alpha, gamma):
    a,g = alpha, gamma
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess

In [ ]:
model_enrolled=lgb.Booster(model_file='model_classifier_enrolled.txt')
model_rebill2=lgb.Booster(model_file='model_classifier_rebill2')
model_visit=lgb.Booster(model_file='model_classifier_visit')
model_rebill4=lgb.Booster(model_file='model_classifier_rebill4')

In [ ]:
dff = df_merged.loc[(df_merged.install_date_utc<'2022-01-24') & (df_merged.app_id=='com.evermatch')].copy()

In [ ]:
df_test = df_merged.loc[(df_merged.install_date_utc>='2022-01-24') & (df_merged.app_id=='com.evermatch')].copy()

In [ ]:
dff['predict_enrolled_origin_mc'] = model_enrolled.predict(dff[features+cat_features])
dff['predict_enrolled_mc'] = dff.apply(lambda x: 1 if sigmoid(x['predict_enrolled_origin_mc']) > 0.5 else 0, axis = 1)

dff['predict_rebill_2_origin'] = model_rebill2.predict(dff[features+cat_features])
dff['predict_rebill_2'] = dff.apply(lambda x: 1 if sigmoid(x['predict_rebill_2_origin']) > 0.5 else 0, axis = 1)

dff['predict_visit_origin'] = model_visit.predict(dff[features+cat_features])
dff['predict_visit'] = dff.apply(lambda x: 1 if sigmoid(x['predict_visit_origin']) > 0.5 else 0, axis = 1)

dff['predict_rebill_4_origin'] = model_rebill4.predict(dff[features+cat_features])
dff['predict_rebill_4'] = dff.apply(lambda x: 1 if sigmoid(x['predict_rebill_4_origin']) > 0.35 else 0, axis = 1)

In [ ]:
df_test['predict_enrolled_origin_mc'] = model_enrolled.predict(df_test[features+cat_features])
df_test['predict_enrolled_mc'] = df_test.apply(lambda x: 1 if sigmoid(x['predict_enrolled_origin_mc']) > 0.5 else 0, axis = 1)

df_test['predict_rebill_2_origin'] = model_rebill2.predict(df_test[features+cat_features])
df_test['predict_rebill_2'] = df_test.apply(lambda x: 1 if sigmoid(x['predict_rebill_2_origin']) > 0.5 else 0, axis = 1)

df_test['predict_visit_origin'] = model_visit.predict(df_test[features+cat_features])
df_test['predict_visit'] = df_test.apply(lambda x: 1 if sigmoid(x['predict_visit_origin']) > 0.5 else 0, axis = 1)

df_test['predict_rebill_4_origin'] = model_rebill4.predict(df_test[features+cat_features])
df_test['predict_rebill_4'] = df_test.apply(lambda x: 1 if sigmoid(x['predict_rebill_4_origin']) > 0.35 else 0, axis = 1)

#### ПРЕДИКТ РЕГРЕССИИ

In [ ]:
class Inapp150:
    def __init__(self, features, cat_features):

        self._target = None

        self._model_reg = LGBMRegressor(
            objective=self._custom_asymmetric_loss,
            metrics=['l2', 'l1'],
            n_estimators=200,
            random_state=20,
            max_depth=6, 
            num_leaves=40, 
            verbose=0
        )
         
        self._df = None

        
        self._features = features
        self._cat_features = cat_features 

    def _custom_asymmetric_loss(self, y_true, y_pred):
        residual = (y_true - y_pred).astype('float')
        grad = np.where(residual > 0, -2 * 1.0 * residual, -2 * 1.0 * residual)
        hess = np.where(residual > 0, 2 * 1.0, 2.0 * 1.0)
        return grad, hess

    def predict_reg(self, dataset):
        dataset['predict_origin'] = self._model_reg.predict(dataset[self._features+self._cat_features])
        dataset['predict'] = dataset.apply(lambda x: max(x['predict_origin'], 0), axis=1)
        dataset['count'] = dataset.apply(lambda x: 1, axis=1)
        
        dataset['predict'] = dataset['predict'] + dataset['app_revenue_0_30']
        
        print('Accuracy {:.4f}'.format(self._model_reg.score(dataset[self._features+self._cat_features], dataset['predict'])))
        return dataset
    
    def fit_reg(self, df, k,target='app_revenue_30_150'):
        self._df = df
        self._target = target
        
        
#         df[self._cat_features] = df[self._cat_features].astype('category')
        
        if k==0:
            self._model_reg.fit(
                    X=self._df[self._features+self._cat_features],
                    y=self._df[self._target]
            )
        else:
            self._model_reg.fit(
                X=self._df[self._features+self._cat_features],
                y=self._df[self._target],
                init_model=self._model_reg
            )
        self._model_reg.booster_.save_model('model_inapp_base_em.txt')
        
        return self._model_reg.feature_importances_

In [ ]:
cat_features2=cat_features

In [ ]:
features2=features.copy()

In [ ]:
features.extend(['predict_enrolled_mc', 'predict_rebill_2', 'predict_rebill_4'])

In [ ]:
dff[features] = dff[features].fillna(0)
df_test[features] = df_test[features].fillna(0)

In [ ]:
dff_train = {n: dff.iloc[n:n+500000, :] 
           for n in range(0, len(dff), 500000)}

In [ ]:
m = Inapp150(features, cat_features2)
for n in dff_train.keys():
    if n==0:
        importance = m.fit_reg(dff_train[n],0)
    else:
        importance = m.fit_reg(dff_train[n],1)

[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
for i in dff_train.keys():
    y = dff_train[i].app_revenue_30_150.copy()
    x = dff_train[i][features2+cat_features2].copy()

    x, x_test, y, y_test = train_test_split(x , y, test_size=0.2, shuffle = False)

    train_data1 = lgb.Dataset(x, label = y)
    test_data1 = lgb.Dataset(x_test, label = y_test)

    lgb_direct_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'num_leaves': 30,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'learning_rate': 0.01,
        'min_data': 20,
        'verbose': 1,
        'eval_freq': 100
    }
    if i==0:
        lgb_model_direct = lgb.train(lgb_direct_params,
                          train_data1,
                          valid_sets=test_data1,
                          num_boost_round=10000,
                          early_stopping_rounds=200,
                          verbose_eval=100
                          )
    else:
        lgb_model_direct = lgb.train(lgb_direct_params,
                          train_data1,
                          valid_sets=test_data1,
                          num_boost_round=10000,
                          early_stopping_rounds=200,
                          verbose_eval=100,
                          init_model=lgb_model_direct         
                          )
    lgb_model_direct.save_model('model_inapp_us_em')
    print("End_direct_",i)


[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084786 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14383
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 150
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Info] Start training from score 0.019871
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.578567
[200]	valid_0's rmse: 0.574725
[300]	valid_0's rmse: 0.582661
Early stopping, best iteration is:
[141]	valid_0's rmse: 0.567327
End_direct_ 0
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Warning] Unknown

In [ ]:
for i in dff_train.keys():    

####################################################################################################################################    
    y = dff_train[i].app_revenue_30_150.copy()
    x = dff_train[i][features2+cat_features2].copy()
    y[y > 0] = 1

    x, x_test, y, y_test = train_test_split(x , y, test_size=0.2, shuffle = False)

    train_data1 = lgb.Dataset(x, label = y)
    test_data1 = lgb.Dataset(x_test, label = y_test)

    lgb_binary_params = {
        'objective': 'binary',
        'metric': 'binary',
        'boosting': 'gbdt',
        'num_leaves': 30,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'learning_rate': 0.01,
        'min_data': 20,
        'verbose': 1,
        'eval_freq': 100
    }

    if i==0:
        lgb_model_binary_pay0 = lgb.train(lgb_binary_params,
                      train_data1,
                      valid_sets=test_data1,
                      num_boost_round=10000,
                      early_stopping_rounds=200,
                      verbose_eval=300
                      )
    else:
        lgb_model_binary_pay0 = lgb.train(lgb_binary_params,
                      train_data1,
                      valid_sets=test_data1,
                      num_boost_round=10000,
                      early_stopping_rounds=200,
                      verbose_eval=300,
                      init_model= lgb_model_binary_pay0                 
                      )
    lgb_model_binary_pay0.save_model('model_inapp_pay0_em')
    
#######################################################

    y = dff_train[i].app_revenue_30_150.copy()
    x = dff_train[i][features2+cat_features2].copy()


    x, x_test, y, y_test = train_test_split(x , y, test_size=0.2, shuffle = False)


    train_data3 = lgb.Dataset(x, label = y)
    test_data3 = lgb.Dataset(x_test, label = y_test)

    lgb_regr_params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'num_leaves': 30,
            'feature_fraction': 0.7,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'learning_rate': 0.01,
            'min_data': 20,
            'verbose': 1,
            'eval_freq': 100
        }

    if i==0:
        lgb_model_regr0 = lgb.train(lgb_regr_params,
                          train_data3,
                          valid_sets=test_data3,
                          num_boost_round=10000,
                          early_stopping_rounds=200,
                          verbose_eval=100
                          )
    else:
        lgb_model_regr0 = lgb.train(lgb_regr_params,
                          train_data3,
                          valid_sets=test_data3,
                          num_boost_round=10000,
                          early_stopping_rounds=200,
                          verbose_eval=100,
                        init_model=lgb_model_regr0
                          )
    lgb_model_regr0.save_model('model_inapp_regr0_em')

[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Info] Number of positive: 241, number of negative: 399759
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100635 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14383
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 150
[LightGBM] [Warning] Unknown parameter: eval_freq
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000602 -> initscore=-7.413820
[LightGBM] [Info] Start training from score -7.413820
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[300]	valid_0's binary_logloss: 0.00108772
[600]	valid_0's binary_logloss: 0.00102388
[900]	valid_0's binary_logloss: 0.00101556
Early stopping, best 